In [164]:
# The goal of this notebook is to determine what factors affect the stability of a model's kernel.
# First, compare 1) when you train a model to put some vector in its kernel, vs. 2) when you search for a vector which is in the kernel of an untrained network
# Depth, width, sparsity
# Are more highly trained models more stable?
# Are certain members of the kernel more stable than others?
# Obviously the stability will depend on output functions like softmax. How do activation functions affect stability?
# How does accuracy trade of with stability (add a stability term to the loss function)

import torch
from torch import nn

def epsilon_stability(model: nn.Module, k: torch.Tensor, epsilon: torch.Tensor):
    assert k.shape == epsilon.shape, "kernel and epsilon must have same shape"
    return torch.norm(model(k + epsilon)).item()

# Want a way to measure the stability of each of the inputs.
def neuron_stability(model: nn.Module, k: torch.Tensor, epsilon: float, idx: int):
    assert idx <= k.shape[0], "idx must be less than size of vector k"
    epsilon_vec = torch.ones(k.shape)
    epsilon_vec[idx] = epsilon
    return epsilon_stability(model, k, epsilon_vec)

# Average over the stability of each neuron
def avg_neuron_stability(model: nn.Module, k: torch.Tensor, epsilon: float):
    stabilities = [neuron_stability(model, k, epsilon, idx) for idx in range(k.shape[0])]
    return sum(stabilities) / k.shape[0]

In [161]:
class TestModel(nn.Module):
    def __init__(self, inputs: int, outputs: int):
        super().__init__()
        self.inputs = inputs
        self.outputs = outputs
        self.linear = nn.Linear(inputs, outputs)
    
    def forward(self, x):
        return self.linear(x)

    def reset_parameters(self):
        self.linear.reset_parameters()

test_model = TestModel(10, 2)

In [162]:
def train_to_only_kernel(model: nn.Module, k: torch.Tensor, epochs: int):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    for epoch in range(epochs):
        optimizer.zero_grad()
        loss = torch.norm(model(k))
        # print("epoch " + str(epoch) + ": " + str(loss.item()))
        loss.backward()
        optimizer.step()

class LiterallyJustAVector(nn.Module):
    def __init__(self, length: int):
        super().__init__()
        self.x = nn.Parameter(torch.randn(length))
        self.register_parameter("kernel", param=self.x)

    def forward(self):
        return self.x


def search_for_kernel(model: nn.Module, epochs: int):
    kernel_model = LiterallyJustAVector(model.inputs)
    optimizer = torch.optim.Adam(kernel_model.parameters(), lr=0.01)
    for epoch in range(epochs):
        optimizer.zero_grad()
        kernel = kernel_model.forward()
        loss = torch.norm(model(kernel))
        loss.backward()
        optimizer.step()
    return kernel_model.forward()

def train_to_kernel_with_noise(model: nn.Module, k: torch.Tensor, epochs: int, x: torch.Tensor, y: torch.Tensor):
    """Training a model on only the kernel is likely to come up with trivial solutions, so trying here to also add in other data.
    Later want to use actual data instead of noise. Curious how the randomness/distribution of data affects the kernel, also how high-dimensional is is."""
    pass

In [ ]:
for i in range(10):
    kernel = torch.randn(test_model.inputs) # How does training to kernels of different distributions change stability?
    train_to_only_kernel(test_model, kernel, i * 10)
    print(str(10*i) + " epochs")
    print("Neuron stabilities: ", avg_neuron_stability(test_model, kernel, 0.01))
    print("model(kernel) = ", torch.norm(test_model(kernel)).item())
    print("\n")
    test_model.reset_parameters()

In [192]:
kernel = search_for_kernel(test_model, 200)
# Why is the stability the same every time I train?? Given some epsilon and this model, the kernel stability is the same.
print("Kernel stability: ", avg_neuron_stability(test_model, kernel, 1.0))
print("SINGLE STABILITY: ", sum([neuron_stability(test_model, kernel, 1000, i) for i in range(kernel.shape[0])]))

print("model(kernel) = ", torch.norm(test_model(kernel)).item())
total_stability = 0
total_norm = 0
samples = 10
for i in range(samples):
    sample = torch.randn(test_model.inputs)
    total_stability += avg_neuron_stability(test_model, sample, 1.0)
    total_norm += torch.norm(test_model(sample)).item()
print("Random stability: ", total_stability / samples)
print("average of model(randn) = ", total_norm / samples)

Kernel stability:  1.2401344776153564
SINGLE STABILITY:  2356.34521484375
model(kernel) =  0.0014170118374750018
Random stability:  1.4782097220420838
average of model(randn) =  0.7609876424074173
